# 小部件事件

深入探讨当小部件值变化时如何处理回调和事件。

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from traitlets import HasTraits

## Traitlets 事件

每个小部件都继承自 traitlets.HasTraits，该类提供了观察值、与其他小部件链接以及值验证的功能。您可以在其 用户指南 中了解更多关于 traitlets 的信息。


每个 `HasTraits` 类都有一个 `observe` 方法，用于观察属性的变化。你可以指定一个 Python 回调函数，当属性变化时会调用该函数。

传递给 `observe` 的回调处理函数将在收到一个变化参数时被调用。变化对象至少包含 `type` 和 `name` 两个键，分别对应通知的类型和触发通知的属性名称。

根据 `type` 的值，可能会传递其他键。当 `type` 为 `change` 时，还会有以下键：

- `owner`：`HasTraits` 实例
- `old`：修改的特征属性的旧值
- `new`：修改的特征属性的新值
- `name`：修改的特征属性的名称

### 在内核中注册回调以处理特征变化

由于 `Widget` 类继承自 `HasTraits`，你可以在模型从前端或其他代码接收到更新时，注册处理程序来处理变化事件。

为了研究这一点，让我们以之前部分的最终示例为基础，稍作修改，去除 `names=True`。

In [ ]:
# Create and display our widgets
slider = widgets.FloatSlider(description='Input:')
square_display = widgets.HTML(description="Square: ", value='{}'.format(slider.value**2))
display(widgets.VBox([slider, square_display]))

# Create function to update square_display's value when slider changes
def update_square_display(change):
    square_display.value = '{}'.format(change.new**2)
    

slider.observe(update_square_display) # removed names="value"



## `names` 参数的重要性

这没有生效！我们的错误信息去哪儿了？

### 在回调中查看错误信息

1. JupyterLab 日志控制台
2. `widgets.Output()`

In [ ]:
# Create and display our widgets
slider = widgets.FloatSlider(description='Input:')
square_display = widgets.HTML(description="Square: ", value='{}'.format(slider.value**2))
display(widgets.VBox([slider, square_display]))


# We use an output widget here for capturing the print calls and showing them at the right place in the Notebook
output = widgets.Output()
display(output)

# Create function to update square_display's value when slider changes
@output.capture()
def update_square_display(change):
    print(change)
    square_display.value = '{}'.format(change.new**2)
    
slider.observe(update_square_display) # removed names="value"


经过调查，我们发现我们收到了 `_property_lock` 特征的通知。这就是为什么使用 `names=` 参数如此重要的原因。

In [ ]:
# Create and display our widgets
slider = widgets.FloatSlider(description='Input:')
square_display = widgets.HTML(description="Square: ", value='{}'.format(slider.value**2))
display(widgets.VBox([slider, square_display]))
output = widgets.Output()
display(output)
@output.capture()
def update_square_display(change):
    print(change)
    square_display.value = '{}'.format(change.new**2)

    
slider.observe(update_square_display, names="value") # added back the names="value"


这会捕获所有值的变化，而不仅仅是通过鼠标进行的变化。

In [ ]:
slider.value = .3

#### `names` 参数的有效值是什么？

任何小部件具有的命名特征都可以被观察。你可以通过 `.traits()` 方法查看小部件具有哪些特征。

In [ ]:
slider.traits()

### `.value` 和验证

大多数 `ipywidgets` 小部件都有一个 `.value` 特征，它对应于它们所表示的数据类型。除了驱动 `observe`，`traitlets` 还会对这些值进行验证和强制转换。因此，如果你尝试将某个值设置为错误的类型，你会收到错误消息，或者该值可能会被强制转换为新的数据类型。

In [ ]:
int_slider = widgets.IntSlider()
float_slider = widgets.FloatSlider()

# fine
float_slider.value = 5.5

# Will get rounded to an int
int_slider.value = 5.5
print(int_slider.value)

# raises an error
int_slider.value = "5.5"

## 练习

使用 `observe` 和 `Output` 小部件，打印出下面 `Textarea` 小部件中的文本字符串的反转。

In [ ]:
text = widgets.Textarea()
output = widgets.Output()

# ...
# ...

display(text, output)

In [ ]:
# %load solutions/observe-reverse.py

### Should you use `observe` or `link`?

何时使用 `observe`

`observe` 在你想对不是小部件的东西产生副作用时最为有用（例如，修改 matplotlib 图表或保存文件），或者当你需要获取前一个值的信息时。

`link` 是将两个小部件的特征连接在一起的最简单方式。你也可以通过将一个函数传递给 `transform` 参数来转换值。


In [ ]:
slider1 = widgets.IntSlider()
slider2 = widgets.IntSlider()
widgets.link((slider1, "value"), (slider2, "value"))

display(slider1, slider2)

你并不限于将 `value` 与 `value` 进行链接。任何具有兼容类型的特征都可以链接。在这个示例中，主滑块的 `min` 值由第二个滑块控制。

In [ ]:
main_slider = widgets.IntSlider(value=3, min=0, max=10, description="main slider")
min_slider = widgets.IntSlider(value=0, min = -10, max=10, description="min slider")
widgets.link((main_slider, "min"), (min_slider, "value"))
display(widgets.VBox([main_slider, min_slider]))

### 练习

使用 `widgets.link` 和 `transform` 参数以及两个函数，使得当修改其中一个文本框时，另一个文本框也会更新。

In [ ]:
def C_to_F(temp):
    return 1.8 * temp + 32

def F_to_C(temp):
    return (temp -32) / 1.8

degree_C = widgets.FloatText(description='Temp $^\circ$C', value=0)
degree_F = widgets.FloatText(description='Temp $^\circ$F', value=C_to_F(degree_C.value))

display(degree_C, degree_F)

In [ ]:
# %load solutions/temperature-link.py

## 高级小部件链接

之前你使用了 `link` 将一个小部件的值与另一个小部件的值链接。

还有一些其他的链接方法，提供了更多的灵活性：

+ `dlink` 是一个 *单向* 链接；更新只会在一个方向上发生，而不会在另一个方向上发生。
+ `jslink` 和 `jsdlink` 在前端进行链接
    - 尽管链接发生在前端，但 Python 对象仍然会更新它们的值。

### 在内核中使用 dlink（即在 Python 中）

第一种方法是使用 `link` 和 `dlink`。这仅在我们与活跃的内核交互时有效。

In [ ]:
caption = widgets.HTML(value='Changes in source values are reflected in target1, but changes in target1 do not affect source')
source, target1 = widgets.IntSlider(description='Source'),\
                  widgets.IntSlider(description='Target 1')

display(caption, source, target1)

dl = widgets.dlink((source, 'value'), (target1, 'value'))


通过调用 `unlink` 可以断开链接。

In [ ]:
dl.unlink()

### 从客户端链接小部件属性

你还可以使用链接小部件在浏览器中直接链接小部件的属性，可以是单向的或双向的。

当将小部件嵌入到没有内核的 HTML 网页中时，JavaScript 链接会保持有效。

In [ ]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
range1, range2 = widgets.IntSlider(description='Range 1'),\
                 widgets.IntSlider(description='Range 2')

display(caption, range1, range2)

l = widgets.jslink((range1, 'value'), (range2, 'value'))

In [ ]:
caption = widgets.Label(value='Changes in source_range values are reflected in target_range1')
source_range, target_range1 = widgets.IntSlider(description='Source range'),\
                              widgets.IntSlider(description='Target range 1')

display(caption, source_range, target_range1)

dl = widgets.jsdlink((source_range, 'value'), (target_range1, 'value'))

通过调用 `unlink` 方法可以断开链接。

In [ ]:
l.unlink()
dl.unlink()

### `link` 和 `jslink` 的区别

**Python 链接**

优点：
1. 允许值的转换

缺点：
1. 性能较差
2. 当内核不运行时不会保持有效

**客户端链接**

优点：
1. 无需内核即可工作
2. 更快的 GUI 更新

缺点：
1. 不支持转换

有关更详细的区别，请参阅 [文档](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#The-difference-between-linking-in-the-kernel-and-linking-in-the-client)。

In [ ]:
leader = widgets.IntSlider(description="leader")
py_follower = widgets.IntSlider(description='python link')
js_follower = widgets.IntSlider(description='client link')

display(leader, js_follower, py_follower)

l_js = widgets.jslink((leader, 'value'), (js_follower, 'value'))
l_py = widgets.link((leader, 'value'), (py_follower, 'value'))


### 连续更新 vs 延迟更新

一些小部件提供了 `continuous_update` 属性，可以选择不断更新值，或者仅在用户提交值时更新（例如，按下 Enter 键或离开控件时）。在下一个示例中，我们看到“延迟”控件只有在用户完成滑动滑块或提交文本框后才会传输其值。而“连续”控件在值发生变化时会持续传输其值。尝试在每个文本框中输入一个两位数，或者拖动每个滑块，以查看它们的区别。

In [ ]:
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

widgets.jslink((a, 'value'), (b, 'value'))
widgets.jslink((a, 'value'), (c, 'value'))
widgets.jslink((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

默认 `continuous_update=True` 的小部件：

- 滑块
- `Text`
- `Textarea`

默认 `continuous_update=False` 的小部件：

- 用于输入数字的文本框（例如 `IntText`）

## 特殊事件

一些小部件，如 `Button`，具有特殊事件，可以在这些事件上挂钩 Python 回调函数。

`Button` 不是用来表示数据类型的。相反，按钮小部件用于处理鼠标点击。`Button` 的 `on_click` 方法可以用来注册一个函数，当按钮被点击时，该函数会被调用。`on_click` 的文档字符串如下所示。

In [ ]:
widgets.Button.on_click?

### 例子

由于按钮点击是无状态的，它们通过自定义消息从前端传输到后端。通过使用 `on_click` 方法，可以显示一个在按钮点击时打印消息的按钮。为了捕获 `print`（或任何其他类型的输出，包括错误）并确保其显示，务必将其发送到 `Output` 小部件（或将你想显示的信息放入 `HTML` 小部件）。

In [ ]:
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

@output.capture()
def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)